In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import cv2
from math import*
import statistics
import numpy as np
from PIL import Image
import matplotlib as plt
from numpy import asarray
from numpy import savetxt
from decimal import Decimal
from collections import Counter
from sklearn.manifold import MDS
from scipy.spatial import distance
from google.colab.patches import cv2_imshow

In [ ]:
InputDir="/content/drive/MyDrive/Distance-Based-Search-Engine /images" # Folder of input images
OutDir="/content/drive/MyDrive/Distance-Based-Search-Engine /FeatureVectors" # Folder of output
size = 5063
ret_size = 100

In [ ]:
cd /content/drive/MyDrive/Distance-Based-Search-Engine /FeatureVectors

/content/drive/MyDrive/IR_ProExc2/FeatureVectors


In [ ]:
h = 256
histograms = np.arange(size*h*3).reshape(size,h,3)
i = 0
for RImgName in os.listdir(InputDir):
  if RImgName == '.ipynb_checkpoints':
    pass
  else:
    InPath=InputDir+"/"+RImgName
    img=cv2.imread(InPath)

    RHist = cv2.calcHist([img],[0],None,[256],[0,256]).reshape(256)
    GHist = cv2.calcHist([img],[1],None,[256],[0,256]).reshape(256)
    BHist = cv2.calcHist([img],[2],None,[256],[0,256]).reshape(256)

    histograms[i,:,0] = RHist
    histograms[i,:,1] = GHist
    histograms[i,:,2] = BHist

    i += 1

In [ ]:
file1 = open(OutDir+"/img_names1.txt","w+")
D_Euclidean = np.arange(size*size).reshape(size,size)
i=0
j = 0
for RImgName in os.listdir(InputDir):
  j = 0
  if i == size:
    break
  file1.writelines(RImgName + '\n')
  for CImgName in os.listdir(InputDir):
    if j > i:
      break
    if i == j:
      D_Euclidean[i,j] = 0 
    else:
      ColHist = histograms[j]
      RowHist = histograms[i]
      D_Euclidean[i,j] = np.linalg.norm(ColHist - RowHist)
      D_Euclidean[j,i] = D_Euclidean[i,j]
    j += 1
  i += 1
file1.close()
savetxt('distance_matrix1.csv', D_Euclidean)

In [ ]:
D = np.loadtxt('distance_matrix1.csv')
print(D.shape)

(5063, 5063)


In [ ]:
D = np.loadtxt('distance_matrix1.csv')

mds = MDS(30,random_state=0)
feature_vectores = mds.fit_transform(D)

savetxt('feature_vectores1.csv', feature_vectores)

/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_mds.py:419: UserWarning: The MDS API has changed. ``fit`` now constructs an dissimilarity matrix from data. To use a custom dissimilarity matrix, set ``dissimilarity='precomputed'``.
  warnings.warn("The MDS API has changed. ``fit`` now constructs an"


In [ ]:
v = np.loadtxt('feature_vectores1.csv')
print(v.shape)

(5063, 30)


In [ ]:
def get_name(num):
  file1 = open(OutDir+"/img_names1.txt","r")
  for i in range (0,size):
    line = file1.readline()
    if i == num:
      file1.close()
      return line
    else:
      pass

def get_number(name):
  i = 0
  file1 = open(OutDir+"/img_names1.txt","r")
  for line in file1:
    dd = line.rstrip('\n').replace(" ", "")
    if dd == name:
      file1.close()
      return i
    else:
      i += 1

##Distance for all queries for mean precision and recall

In [ ]:
queries = []
Qfile = open("/content/drive/MyDrive/Distance-Based-Search-Engine /Query/queries.txt","r")
for line in Qfile:
  name = line.rstrip('\n').replace(" ", "")
  queries.append(name)
Qfile.close()

In [ ]:
def argsort(seq):
    return sorted(range(len(seq)), key=seq.__getitem__)

###Eucleadian Distance


In [ ]:
path = "/content/drive/MyDrive/Distance-Based-Search-Engine /ground_truth"
v = np.loadtxt('feature_vectores1.csv')

In [ ]:
EuclideanDistance = []
num_of_good_imgs = []
precision = []
recall = []
query_set = []
k = 0

for input_name in queries:
  tp = 0
  k += 1
  input_number = get_number(input_name + ".jpg")
  query_vect = v[input_number]
  
  for l in range(size):
    img_vect = v[l]
    EuclideanDistance.append(distance.euclidean(query_vect, img_vect))

  EuclideanDistance_min = argsort(EuclideanDistance)[:ret_size] #give us index of k minimum distance

  file2 = open(path + input_name + ".txt","r")
  for line in file2:
    dd = line.rstrip('\n').replace(" ", "")
    query_set.append(dd + '.jpg')
  num_of_good_imgs = len(query_set)
  for i in EuclideanDistance_min:
    img_name = get_name(i).rstrip()
    for j in query_set:
      if img_name == j:
        tp += 1
      else:
        pass

  file2.close()
  precision.append(tp / ret_size)
  recall.append(tp / num_of_good_imgs)
  EuclideanDistance.clear()
  query_set.clear()

precision_mean = statistics.mean(precision)
recall_mean = statistics.mean(recall)

print(precision)
print(recall)
print("\n Mean Precision: " ,precision_mean)
print("\n Mean Recall: " ,recall_mean)

[0.02, 0.01, 0.04, 0.01, 0.02, 0.01, 0.01, 0.0, 0.02, 0.01, 0.01, 0.0, 0.0, 0.01, 0.0, 0.03, 0.0, 0.03, 0.01, 0.01, 0.01, 0.02, 0.01, 0.03, 0.0, 0.01, 0.0, 0.01, 0.01, 0.01, 0.01, 0.02, 0.01, 0.02, 0.02, 0.01, 0.01, 0.02, 0.01, 0.01, 0.01, 0.0, 0.03, 0.0, 0.01, 0.01, 0.01, 0.01, 0.0, 0.0, 0.02, 0.05, 0.03, 0.01, 0.09]
[0.08333333333333333, 0.041666666666666664, 0.16666666666666666, 0.041666666666666664, 0.08333333333333333, 0.08333333333333333, 0.08333333333333333, 0.0, 0.16666666666666666, 0.08333333333333333, 0.2, 0.0, 0.0, 0.2, 0.0, 0.23076923076923078, 0.0, 0.23076923076923078, 0.07692307692307693, 0.07692307692307693, 0.0196078431372549, 0.0392156862745098, 0.0196078431372549, 0.058823529411764705, 0.0, 0.2, 0.0, 0.2, 0.2, 0.2, 0.02857142857142857, 0.05714285714285714, 0.02857142857142857, 0.05714285714285714, 0.05714285714285714, 0.16666666666666666, 0.16666666666666666, 0.3333333333333333, 0.16666666666666666, 0.16666666666666666, 0.07692307692307693, 0.0, 0.23076923076923078, 0

###Cosine Distance

In [ ]:
CosineDistance = []
num_of_good_imgs2 = []
precision2 = []
recall2 = []
query_set2 = []
k = 0
for input_name in queries:
  tp = 0
  k += 1
  input_number = get_number(input_name + ".jpg")
  query_vect = v[input_number]
  
  for l in range(size):
    img_vect = v[l]
    CosineDistance.append(distance.cosine(query_vect, img_vect))

  CosineDistance_min= argsort(CosineDistance)[:ret_size] #give us index of k minimum distance

  file2 = open(path + input_name + ".txt","r")
  for line in file2:
    dd = line.rstrip('\n').replace(" ", "")
    query_set2.append(dd + '.jpg')
  num_of_good_imgs2 = len(query_set2)
  for i in CosineDistance_min:
    img_name = get_name(i).rstrip()
    for j in query_set2:
      if img_name == j:
        tp += 1
      else:
        pass

  file2.close()
  precision2.append(tp / ret_size)
  recall2.append(tp / num_of_good_imgs2)
  CosineDistance.clear()
  query_set2.clear()

precision_mean2 = statistics.mean(precision2)
recall_mean2 = statistics.mean(recall2)

print("Precisions: ", precision2)
print("Recalls: ", recall2)
print("\n Mean Precision: " ,precision_mean2)
print("\n Mean Recall: " ,recall_mean2)

Precisions:  [0.02, 0.01, 0.04, 0.0, 0.02, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.0, 0.01, 0.01, 0.0, 0.02, 0.0, 0.02, 0.01, 0.01, 0.02, 0.02, 0.01, 0.02, 0.0, 0.01, 0.0, 0.01, 0.01, 0.01, 0.01, 0.02, 0.03, 0.02, 0.03, 0.01, 0.01, 0.02, 0.01, 0.01, 0.02, 0.0, 0.03, 0.0, 0.01, 0.01, 0.01, 0.01, 0.0, 0.0, 0.02, 0.03, 0.03, 0.01, 0.05]
Recalls:  [0.08333333333333333, 0.041666666666666664, 0.16666666666666666, 0.0, 0.08333333333333333, 0.08333333333333333, 0.08333333333333333, 0.08333333333333333, 0.08333333333333333, 0.08333333333333333, 0.2, 0.0, 0.2, 0.2, 0.0, 0.15384615384615385, 0.0, 0.15384615384615385, 0.07692307692307693, 0.07692307692307693, 0.0392156862745098, 0.0392156862745098, 0.0196078431372549, 0.0392156862745098, 0.0, 0.2, 0.0, 0.2, 0.2, 0.2, 0.02857142857142857, 0.05714285714285714, 0.08571428571428572, 0.05714285714285714, 0.08571428571428572, 0.16666666666666666, 0.16666666666666666, 0.3333333333333333, 0.16666666666666666, 0.16666666666666666, 0.15384615384615385, 0.0, 0

###Minkowski Distance

In [ ]:
MinkowskiDistance = []
num_of_good_imgs3 = []
precision3 = []
recall3 = []
query_set3 = []
k = 0
for input_name in queries:
  tp = 0
  k += 1
  input_number = get_number(input_name + ".jpg")
  query_vect = v[input_number]
  
  for l in range(size):
    img_vect = v[l]
    MinkowskiDistance.append(distance.minkowski(query_vect, img_vect))

  MinkowskiDistance_min= argsort(MinkowskiDistance)[:ret_size] #give us index of k minimum distance

  file2 = open(path + input_name + ".txt","r")
  for line in file2:
    dd = line.rstrip('\n').replace(" ", "")
    query_set3.append(dd + '.jpg')
  num_of_good_imgs3 = len(query_set3)
  for i in MinkowskiDistance_min:
    img_name = get_name(i).rstrip()
    for j in query_set3:
      if img_name == j:
        tp += 1
      else:
        pass

  file2.close()
  precision3.append(tp / ret_size)
  recall3.append(tp / num_of_good_imgs3)
  MinkowskiDistance.clear()
  query_set3.clear()

precision_mean3 = statistics.mean(precision3)
recall_mean3 = statistics.mean(recall3)

print("Precisions: ", precision3)
print("Recalls: ", recall3)
print("\n Mean Precision: " ,precision_mean3)
print("\n Mean Recall: " ,recall_mean3)

Precisions:  [0.02, 0.01, 0.04, 0.01, 0.02, 0.01, 0.01, 0.0, 0.02, 0.01, 0.01, 0.0, 0.0, 0.01, 0.0, 0.03, 0.0, 0.03, 0.01, 0.01, 0.01, 0.02, 0.01, 0.03, 0.0, 0.01, 0.0, 0.01, 0.01, 0.01, 0.01, 0.02, 0.01, 0.02, 0.02, 0.01, 0.01, 0.02, 0.01, 0.01, 0.01, 0.0, 0.03, 0.0, 0.01, 0.01, 0.01, 0.01, 0.0, 0.0, 0.02, 0.05, 0.03, 0.01, 0.09]
Recalls:  [0.08333333333333333, 0.041666666666666664, 0.16666666666666666, 0.041666666666666664, 0.08333333333333333, 0.08333333333333333, 0.08333333333333333, 0.0, 0.16666666666666666, 0.08333333333333333, 0.2, 0.0, 0.0, 0.2, 0.0, 0.23076923076923078, 0.0, 0.23076923076923078, 0.07692307692307693, 0.07692307692307693, 0.0196078431372549, 0.0392156862745098, 0.0196078431372549, 0.058823529411764705, 0.0, 0.2, 0.0, 0.2, 0.2, 0.2, 0.02857142857142857, 0.05714285714285714, 0.02857142857142857, 0.05714285714285714, 0.05714285714285714, 0.16666666666666666, 0.16666666666666666, 0.3333333333333333, 0.16666666666666666, 0.16666666666666666, 0.07692307692307693, 0.0,

###Jaccard Distance

In [ ]:
JaccardDistance = []
num_of_good_imgs4 = []
precision4 = []
recall4 = []
query_set4 = []
k = 0
for input_name in queries:
  tp = 0
  k += 1
  input_number = get_number(input_name + ".jpg")
  query_vect = v[input_number]
  
  for l in range(size):
    img_vect = v[l]
    JaccardDistance.append(distance.jaccard(query_vect, img_vect))

  JaccardDistance_min= argsort(JaccardDistance)[:ret_size] #give us index of k minimum distance

  file2 = open(path + input_name + ".txt","r")
  for line in file2:
    dd = line.rstrip('\n').replace(" ", "")
    query_set4.append(dd + '.jpg')

  num_of_good_imgs4 = len(query_set4)
  
  for i in JaccardDistance_min:
    img_name = get_name(i).rstrip()
    for j in query_set4:
      if img_name == j:
        tp += 1
      else:
        pass

  file2.close()
  precision4.append(tp / ret_size)
  recall4.append(tp / num_of_good_imgs4)
  JaccardDistance.clear()
  query_set4.clear()

precision_mean4 = statistics.mean(precision4)
recall_mean4 = statistics.mean(recall4)

print("Precisions: ", precision4)
print("Recalls: ", recall4)
print("\n Mean Precision: " ,precision_mean4)
print("\n Mean Recall: " ,recall_mean4)

Precisions:  [0.01, 0.01, 0.01, 0.0, 0.01, 0.01, 0.01, 0.0, 0.01, 0.0, 0.01, 0.0, 0.0, 0.01, 0.0, 0.01, 0.0, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.0, 0.01, 0.0, 0.01, 0.01, 0.01, 0.02, 0.02, 0.02, 0.02, 0.02, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.0, 0.01, 0.0, 0.01, 0.01, 0.01, 0.01, 0.0, 0.0, 0.02, 0.02, 0.02, 0.01, 0.02]
Recalls:  [0.041666666666666664, 0.041666666666666664, 0.041666666666666664, 0.0, 0.041666666666666664, 0.08333333333333333, 0.08333333333333333, 0.0, 0.08333333333333333, 0.0, 0.2, 0.0, 0.0, 0.2, 0.0, 0.07692307692307693, 0.0, 0.07692307692307693, 0.07692307692307693, 0.07692307692307693, 0.0196078431372549, 0.0196078431372549, 0.0196078431372549, 0.0196078431372549, 0.0, 0.2, 0.0, 0.2, 0.2, 0.2, 0.05714285714285714, 0.05714285714285714, 0.05714285714285714, 0.05714285714285714, 0.05714285714285714, 0.16666666666666666, 0.16666666666666666, 0.16666666666666666, 0.16666666666666666, 0.16666666666666666, 0.07692307692307693, 0.0, 0.07692307692307693, 0.0, 0.0769

##Code to retrieve and show relevant images

###Enter query image

In [ ]:
retrieve_size = 20

query_name = "christ_church_000179"
query_number = get_number(query_name + ".jpg")
query_vector = v[query_number]

query_file = open(path + query_name + ".txt","r")
relevant_images = []
for line in query_file:
    dd = line.rstrip('\n').replace(" ", "")
    relevant_images.append(dd + '.jpg')
query_file.close()

###Euclidean Distance

####Retrieve Images

In [ ]:
EuclideanDistance = np.arange(size)
for i in range(size):
  img_vect = v[i]
  EuclideanDistance[i] = distance.euclidean(query_vector, img_vect)

EuclideanDistance_min = EuclideanDistance.argsort()[:retrieve_size] #give us index of k minimum distance

for i in EuclideanDistance_min:
  img_name = get_name(i).rstrip()
  print(img_name + '\n')
  img_path = InputDir + img_name
  img = cv2.imread(img_path, cv2.IMREAD_UNCHANGED)
  cv2_imshow(img)
  print('\n...........................................................................')

####Calculate Precision and Recall

In [ ]:
euc_TP = 0
for i in EuclideanDistance_min:
  img_name = get_name(i).rstrip()
  for j in relevant_images:
    if img_name == j:
      euc_TP += 1
    else:
      pass
euc_precision = euc_TP/retrieve_size
euc_recall = euc_TP/len(relevant_images)
print("Precision: ", euc_precision)
print("Recall: :", euc_recall)

Precision:  0.05
Recall: : 0.0196078431372549


###Cosine Distance

####Retrieve Images

In [ ]:
CosineDistance = np.arange(size)
for i in range(size):
  img_vect = v[i]
  CosineDistance[i] = distance.cosine(query_vector, img_vect)

CosineDistance_min = CosineDistance.argsort()[:retrieve_size] #give us index of k minimum distance

for i in CosineDistance_min:
  img_name = get_name(i).rstrip()
  print(img_name + '\n')
  img_path = InputDir + img_name
  img = cv2.imread(img_path, cv2.IMREAD_UNCHANGED)
  cv2_imshow(img)
  print('\n...........................................................................')

####Calculate Precision and Recall

In [ ]:
cos_TP = 0
for i in CosineDistance_min:
  img_name = get_name(i).rstrip()
  for j in relevant_images:
    if img_name == j:
      cos_TP += 1
    else:
      pass
cos_precision = cos_TP/retrieve_size
cos_recall = cos_TP/len(relevant_images)
print("Precision: ", cos_precision)
print("Recall: ", cos_recall)

Precision:  0.0
Recall:  0.0


###Minkowski Distance

####Retrieve Images

In [ ]:
MinkowskiDistance = np.arange(size)
for i in range(size):
  img_vect = v[i]
  MinkowskiDistance[i] = distance.minkowski(query_vector, img_vect)

MinkowskiDistance_min = MinkowskiDistance.argsort()[:retrieve_size] #give us index of k minimum distance

for i in MinkowskiDistance_min:
  img_name = get_name(i).rstrip()
  print(img_name + '\n')
  img_path = InputDir + img_name
  img = cv2.imread(img_path, cv2.IMREAD_UNCHANGED)
  cv2_imshow(img)
  print('\n...........................................................................')

####Calculate Precision and Recall

In [ ]:
min_TP = 0
for i in MinkowskiDistance_min:
  img_name = get_name(i).rstrip()
  for j in relevant_images:
    if img_name == j:
      min_TP += 1
    else:
      pass
min_precision = min_TP/retrieve_size
min_recall = min_TP/len(relevant_images)
print("Precision: ", min_precision)
print("Recall: ", min_recall)

Precision:  0.05
Recall:  0.0196078431372549


###Jaccard Distance

####Retrieve Images

In [ ]:
JaccardDistance = np.arange(size)
for i in range(size):
  img_vect = v[i]
  JaccardDistance[i] = distance.jaccard(query_vector, img_vect)

JaccardDistance_min = JaccardDistance.argsort()[:retrieve_size] #give us index of k minimum distance

for i in JaccardDistance_min:
  img_name = get_name(i).rstrip()
  print(img_name + '\n')
  img_path = InputDir + img_name
  img = cv2.imread(img_path, cv2.IMREAD_UNCHANGED)
  cv2_imshow(img)
  print('\n...........................................................................')

####Calculate Precision and Recall

In [ ]:
jac_TP = 0
for i in JaccardDistance_min:
  img_name = get_name(i).rstrip()
  for j in relevant_images:
    if img_name == j:
      jac_TP += 1
    else:
      pass
jac_precision = jac_TP/retrieve_size
jac_recall = jac_TP/len(relevant_images)
print("Precision: ", jac_precision)
print("Recall: ", jac_recall)

Precision:  0.05
Recall:  0.0196078431372549
